In [1]:
!pip3 install sentencepiece
!pip3 install tf-sentencepiece
#!pip install tensorflow==1.13.1

In [2]:
import os, sys
from google.colab import drive
drive.mount('/content/gdrive')
gitDir = "/content/gdrive/My Drive/nlp/"
os.chdir(gitDir + "data/")
print(os.listdir("."))

#sys.path.insert(0,gitDir + "data")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
['Kaggle Amazon Fine Food.ipynb', '.ipynb_checkpoints', 'Organic Dataset.ipynb', 'cache', '__pycache__', 'Amazon Reviews.ipynb', 'Amazon Multilingual.ipynb', 'tmpBACKUP.py', 'tmp.ipynb', 'Dataset_Amazon_Multilingual.ipynb', 'amazon_multilingual.py']


In [3]:
import tensorflow as tf
print(tf.__version__)
import tensorflow_hub as hub
import numpy as np
import tf_sentencepiece

import amazon_multilingual
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

from tqdm import tqdm

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

W0518 16:41:48.547347 140244777322368 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


1.13.1
Version:  1.13.1
Eager mode:  False
Hub version:  0.4.0
GPU is available


## create graph

In [0]:
batchSize = 5

In [5]:
dataset_train, feed_dict_train, length_train = amazon_multilingual.getData("TEST", shuffle=True, buffer=batchSize, batchsize=batchSize)

cache/amazon_multilingual_TEST_category.npy and cache/amazon_multilingual_TEST_labels.npy  exist. Using saved npy files...


In [6]:
dataset_val, feed_dict_val, length_val = amazon_multilingual.getData("TEST", shuffle=False, buffer=batchSize, batchsize=batchSize)

cache/amazon_multilingual_TEST_category.npy and cache/amazon_multilingual_TEST_labels.npy  exist. Using saved npy files...
dataset is not shuffled and prefetched


In [7]:
iterator = tf.data.Iterator.from_structure(dataset_train.output_types, dataset_train.output_shapes)
train_iterator = iterator.make_initializer(dataset_train)
val_iterator = iterator.make_initializer(dataset_val)

Instructions for updating:
Colocations handled automatically by placer.


W0518 16:41:49.378643 140244777322368 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/iterator_ops.py:358: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


In [0]:
class Model:
    def __init__(self, data_X, data_Y, xling=None):
        self.n_class = 35
        if xling is None:
          print("Downloading xling...")
          self.xling = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1", trainable=False) 
        else:
          self.xling = xling
        self.create_architecture(data_X, data_Y)
        
    def create_architecture(self, data_X, data_Y):
        y_hot = tf.one_hot(data_Y, depth=self.n_class)
        self.logits = self.forward(data_X)
        
        self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_hot, logits=self.logits))
        self.train_op = tf.train.AdamOptimizer().minimize(self.loss)

        self.predictions = tf.argmax(self.logits,1)
        #self.acc, self.acc_op = tf.metrics.accuracy(labels=data_Y, predictions=self.predictions)
        
        #a = tf.cast(self.predictions, tf.float64)
        self.accuracy = tf.reduce_mean( tf.cast(tf.equal(self.predictions, tf.cast(data_Y, tf.int64)), tf.float32) )
        
    def forward(self, X):
        embedded_text = self.xling(text_input)
        output = tf.layers.dense(embedded_text, self.n_class)
        return output

# train

In [0]:
xling = hub.Module("https://tfhub.dev/google/universal-sentence-encoder-xling-many/1", trainable=False)

In [10]:
text_input, label = iterator.get_next()
model = Model(text_input, label, xling)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0518 16:42:10.416462 140244777322368 saver.py:1483] Saver not created because there are no variables in the graph to restore


Instructions for updating:
Use keras.layers.dense instead.


W0518 16:42:11.187274 140244777322368 deprecation.py:323] From <ipython-input-8-5426162c907b>:26: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [0]:
init_op = tf.group([tf.local_variables_initializer(), tf.global_variables_initializer(), tf.tables_initializer()])
sess = tf.Session()
sess.run(init_op)

In [14]:
%%time

epochs = 5

for epoch in range(epochs):
  train_loss, train_accuracy = 0, 0
  val_loss, val_accuracy = 0, 0
  
  
  sess.run(train_iterator, feed_dict=feed_dict_train)
  
  try:
    with tqdm(total = length_train) as pbar:
      while True:
        _, a, l = sess.run([model.train_op, model.accuracy, model.loss])
        #print(a,l)
        train_loss += l
        train_accuracy += a
        pbar.update(batchSize)
  except tf.errors.OutOfRangeError:
     pass
     #print("out!")
      
      
  
  # Validation
  sess.run(val_iterator, feed_dict=feed_dict_val) 
  try:
    with tqdm(total = length_val) as pbar:
      while True:
        _, a, l = sess.run([model.train_op, model.accuracy, model.loss])
        val_loss += l
        val_accuracy += a
        pbar.update(batchSize)
  except tf.errors.OutOfRangeError:
     pass
     #print("out!")
      
      
      
  print('\nEpoch: {}'.format(epoch + 1))
  print('Train accuracy: {:.4f}, loss: {:.4f}'.format(train_accuracy / length_train,train_loss / length_train))
  print('Val accuracy: {:.4f}, loss: {:.4f}\n'.format(val_accuracy / length_val, val_loss / length_val))


  0%|          | 0/15 [00:00<?, ?it/s]


Epoch: 1
Train accuracy: 0.0800, loss: 0.6066
Val accuracy: 0.0800, loss: 0.5983


Epoch: 2
Train accuracy: 0.0800, loss: 0.5898
Val accuracy: 0.0800, loss: 0.5818


Epoch: 3
Train accuracy: 0.0800, loss: 0.5737
Val accuracy: 0.0800, loss: 0.5663



100%|██████████| 15/15 [00:00<00:00, 500.07it/s]


Epoch: 4
Train accuracy: 0.0800, loss: 0.5587
Val accuracy: 0.0800, loss: 0.5517


Epoch: 5
Train accuracy: 0.0800, loss: 0.5450
Val accuracy: 0.0800, loss: 0.5380

CPU times: user 429 ms, sys: 56.8 ms, total: 486 ms
Wall time: 378 ms


In [107]:
fd = {text_input: ["eins","eins","eins"], label: [0,30,30] }

p, a, acc = sess.run([model.predictions, model.acc_op, model.acc], feed_dict=fd)
print(p,a, acc)

#p, a = sess.run([model.predictions, model.accuracy], feed_dict=fd)
#print(p,a)

p, a = sess.run([model.predictions, model.accuracy], feed_dict=fd)
print(a)

[30 30 30] 0.4 0.25
1.0


## save model
see https://stackoverflow.com/questions/33759623/tensorflow-how-to-save-restore-a-model

https://cv-tricks.com/tensorflow-tutorial/save-restore-tensorflow-models-quick-complete-tutorial/

In [21]:
!ls -alh /content/testsave

total 161M
drwxr-xr-x 3 root root 4.0K May 18 15:14 .
drwxr-xr-x 1 root root 4.0K May 18 15:14 ..
-rw-r--r-- 1 root root 161M May 18 15:14 saved_model.pb
drwxr-xr-x 2 root root 4.0K May 18 15:14 variables


In [20]:
tf.saved_model.simple_save(sess, "/content/testsave", inputs={"text_input": text_input, "label": label}, outputs={"prediction": model.predictions})

INFO:tensorflow:Assets added to graph.


I0518 15:14:37.942099 139736002389888 builder_impl.py:654] Assets added to graph.


INFO:tensorflow:No assets to write.


I0518 15:14:37.946287 139736002389888 builder_impl.py:449] No assets to write.


INFO:tensorflow:SavedModel written to: /content/testsave/saved_model.pb


I0518 15:14:55.698210 139736002389888 builder_impl.py:414] SavedModel written to: /content/testsave/saved_model.pb


In [28]:
!ls -alh /content/saver

total 606M
drwxr-xr-x 2 root root 4.0K May 18 15:19 .
drwxr-xr-x 1 root root 4.0K May 18 15:19 ..
-rw-r--r-- 1 root root  107 May 18 15:19 checkpoint
-rw-r--r-- 1 root root 444M May 18 15:19 model.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root  46K May 18 15:19 model.ckpt.index
-rw-r--r-- 1 root root 163M May 18 15:19 model.ckpt.meta


In [27]:
saver = tf.train.Saver()
saver.save(sess, "/content/saver/model.ckpt")

'/content/saver/model.ckpt'